# 豆瓣

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import random

user_agents = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]

# user-agent
request_header = {
    "User-Agent":
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 "
        "Safari/537.36 Edg/114.0.1823.58",
    "Connection":
        "keep-alive",
    "Referer":
        "https://www.douban.com"
}
url_pool = []

### mysql数据库

In [2]:
import pymysql
# 创建连接
cnx = pymysql.connect(host='localhost', user='root', password='123qwe12')
cursor = cnx.cursor()

def create_database():
    # 创建数据库
    cursor.execute("CREATE DATABASE IF NOT EXISTS IR_db")
    # 使用数据库
    cnx.select_db('IR_db')

def create_table():
    # 创建表
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS douban (
            id INT AUTO_INCREMENT PRIMARY KEY,
            title VARCHAR(255),
            content TEXT,
            url VARCHAR(255),
            links TEXT
        )
    """)
    # 提交
    cnx.commit()

create_database()
create_table()

In [3]:
def check_duplicate(url):
    # 查询数据
    cursor.execute("""
        SELECT * FROM douban WHERE url = %s
    """, (url,))
    if cursor.fetchone() is None:
        return False
    else:
        return True

def save_data(title,content,url,links):
    if check_duplicate(url):
        return
    # 插入数据
    cursor.execute("""
        INSERT INTO douban (title, content, url, links) VALUES (%s, %s, %s, %s)
    """, (title, content, url, links))
    # 提交
    cnx.commit()


### 处理网页外链

In [4]:
def deal_review_content(url):
    resp = requests.get(url, headers=request_header)
    status_code = resp.status_code
    if status_code == 200:
        html_text = resp.text
        # 解析html
        soup = BeautifulSoup(html_text, "html.parser")
        article = soup.select('#content > div > div.article')[0]
        title = article.select('h1 > span')[0].text
        content_div = article.select('div.main-bd > div > div.review-content.clearfix')[0]
        content = content_div.text
        a = soup.select('#content > div > div.aside > div > div.subject-title > a')[0]
        link = a.get('href')
        # 去掉链接中的查询参数
        link = link.split('?')[0]
        save_data(title, content, url, link)                
    else:
        print('请求失败')
        

def deal_review(detail_soup):
    review_items = detail_soup.select('div.review-list > div > div > div.main-bd')
    links = []
    for item in review_items:
        a = item.select('h2 > a')[0]
        url = a.get('href')
        # 去掉链接中的查询参数
        url = url.split('?')[0]
        links.append(url)
        # 直接保存影评内容
        deal_review_content(url)
    return links


def deal_recommend(detail_soup):
    dls = detail_soup.select('#db-rec-section > div > dl')
    links = []
    for dl in dls:
        a = dl.select('dd > a')[0]
        url = a.get('href')
        # 去掉链接中的查询参数
        url = url.split('?')[0]
        # 推荐影片放入url池
        if url not in url_pool:
            url_pool.append(url)
        links.append(url)
    return links

def deal_recommend_book(detail_soup):
    dls = detail_soup.select('#recommendations > div > dl')
    links = []
    for dl in dls:
        a = dl.select('dd > a')[0]
        url = a.get('href')
        # 去掉链接中的查询参数
        url = url.split('?')[0]
        # 推荐影片放入url池
        if url not in url_pool:
            url_pool.append(url)
        links.append(url)
    return links

### 处理电影详情页

In [5]:
import random

def deal_hd(hd_div):
    homepage_a = hd_div.select('div.hd > a')[0]
    homepage_url = homepage_a.get('href').strip()
    if check_duplicate(homepage_url) is False:
        url_pool.append(homepage_url)
        while len(url_pool) > 0:
            deal_url()
    else:
        print(f"已经处理过url：{homepage_url}") 


def deal_url_book():
    homepage_url = url_pool.pop(0)
    try:
        request_header['User-Agent'] = random.choice(user_agents)
        response = requests.get(url=homepage_url, headers=request_header)
        status = response.status_code
    except:
        print(f"请求失败{homepage_url}")
        request_header['User-Agent'] = random.choice(user_agents)
        return
    
    if status == 200:
        try:
            # 进入详情页
            detail_soup = BeautifulSoup(response.text, 'html.parser')
            # 获取标题
            title = detail_soup.select('#wrapper > h1 > span')[0].getText().strip()
            # 获取网页内容 content = info+intro+author+comment
            info = detail_soup.select('#info')[0].getText().strip()
            intro_div = detail_soup.select('div#link-report div.intro')[0]
            intro = [' '.join(p.text.split()) for p in intro_div.select('p')]
            intro = "\n".join(intro)
            author_div = detail_soup.select('div.indent div.intro')[len(detail_soup.select('div.indent div.intro'))-1]
            author = [' '.join(p.text.split()) for p in author_div.select('p')]
            author = "\n".join(author)
            comment_lis = detail_soup.select('div#score > div > ul > li.comment-item')
            comments = []
            for comment_li in comment_lis:
                comment = comment_li.select('div.comment > p > span')[0].getText().strip()
                comments.append(comment)
            comments = "\n".join(comments)
            content = "\n".join([info, intro, author, comments])
        except:
            print(f"解析失败{homepage_url}")
            return
        try:
            # 处理网页外链
            links_review = deal_review(detail_soup)
            links_recommend = deal_recommend_book(detail_soup)
            links = "\n".join(links_review + links_recommend)
        except:
            print(f"解析外链失败{homepage_url}")
            return
        try:
            # 保存数据
            save_data(title, content, homepage_url, links)
        except:
            print(f"保存失败{homepage_url}")
            return
    else:
        print("状态码非200")


def deal_url():
    homepage_url = url_pool.pop(0)
    try:
        request_header['User-Agent'] = random.choice(user_agents)
        response = requests.get(url=homepage_url, headers=request_header)
        status = response.status_code
    except:
        print(f"请求失败{homepage_url}")
        request_header['User-Agent'] = random.choice(user_agents)
        return
    
    if status == 200:
        try:
            # 进入详情页
            detail_soup = BeautifulSoup(response.text, 'html.parser')
            # 获取标题
            title = detail_soup.select('#content > h1 > span')[0].getText().strip()
            # 获取网页内容 content = info+summary+comment
            info = detail_soup.select('#info')[0].getText().strip()
            summary_spans = detail_soup.select('div#link-report-intra span[property="v:summary"]')
            summary_t = [' '.join(span.getText().split()) for span in summary_spans]
            summary = "\n".join(summary_t)
            comment_divs = detail_soup.select('#hot-comments > div.comment-item')
            comments = []
            for comment_div in comment_divs:
                comment = comment_div.select('div.comment > p > span')[0].getText().strip()
                comments.append(comment)
            comments = "\n".join(comments)
            content = "\n".join([info, summary, comments])
        except:
            print(f"解析失败{homepage_url}")
            return
        try:
            # 处理网页外链
            links_review = deal_review(detail_soup)
            links_recommend = deal_recommend(detail_soup)
            links = "\n".join(links_review + links_recommend)
        except:
            print(f"解析外链失败{homepage_url}")
            return
        try:
            # 保存数据
            save_data(title, content, homepage_url, links)
        except:
            print(f"保存失败{homepage_url}")
            return
    else:
        print("状态码非200")


### 进入top250主页

In [6]:
# 解析top250主页，what = movie/book
def search_in_top250(what):
    for i in range(0,250,25):
        # 生成url
        url = "https://{}.douban.com/top250?start={}".format(what,i)
        # 发送请求
        resp = requests.get(url,headers=request_header)
        status_code = resp.status_code
        if status_code == 200:
            html_text = resp.text
            # 解析html
            soup = BeautifulSoup(html_text,"lxml")
            if what == 'movie':
                # 获取到这25个电影的链接（存在div.hd里）
                hd_divs = soup.select('div.item > div.info > div.hd')
                for hd_div in hd_divs:
                    deal_hd(hd_div)
            else:
                # 获取到这25个书籍的链接（存在div.pl2里）
                pl2_divs = soup.select('tr.item > td > div.pl2')
                for pl2_div in pl2_divs:
                    a = pl2_div.select('a')[0]
                    url = a.get('href').strip()
                    if check_duplicate(url) is False:
                        url_pool.append(url)
                        while len(url_pool) > 0:
                            deal_url_book()
                    else:
                        print(f"已经处理过url：{url}")
        else:
            print("Error in top250: {}".format(status_code))

In [ ]:
search_in_top250('book')